In [ ]:
#| default_exp shared_actions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections.abc import MutableMapping
from typing import Any, Dict
from fastcore.foundation import patch
import logging

# shared_context
> Informations sharing between actions

We should facilitate the sharing of information between actions. This is a common need in many applications, and we can achieve this by using a shared context.

To achieve this, we can create a `SharedContext` class that will hold the shared information. This class will be responsible for storing and retrieving the information as needed.

In [ ]:
#| exporti

logger = logging.getLogger("TriggerKit-SharedContext")

In [ ]:
#| export

class SharedContext(MutableMapping):
    """
    A dictionary-like object that allows sharing of context between different parts of a program.
    It is designed to be used as a singleton, ensuring that all parts of the program share the same context.
    """
    def __init__(self, base: Dict[str, Any] = None):
        self._data = base or {}
        self._writes = {} # Record of writes during this run

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        self._data[key] = value
        self._writes[key] = value
        logger.debug(f"[context] Set key '{key}' = {repr(value)}")

    def __delitem__(self, key):
        del self._data[key]
        self._writes[key] = None
        logger.debug(f"[context] Deleted key '{key}'")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def get(self, key, default=None):
        return self._data.get(key, default)

    def to_dict(self) -> Dict[str, Any]:
        return dict(self._data)
    
    def __setattr__(self, key, value):
        logger.debug(f"[context] Set key '{key}' = {repr(value)}")
        if key in {"_data", "_writes"}:
            super().__setattr__(key, value)
        else:
            self._data[key] = value
            self._writes[key] = value

    def __getattr__(self, key):
        return self._data.get(key)


To make it easy to use, the class should be access like a dictionary, allowing us to set and get values using the `[]` operator. This will make it easy to share information between actions without having to pass it around explicitly.

In [ ]:
#| export

@patch
def __contains__(self: SharedContext, key):
    return key in self._data

@patch
def __repr__(self: SharedContext):
    return f"SharedContext({self._data})"

@patch
def __str__(self: SharedContext):
    return f"SharedContext({self._data})"

@patch
def __eq__(self: SharedContext, other):
    if isinstance(other, SharedContext):
        return self._data == other._data
    return False

@patch
def __ne__(self: SharedContext, other):
    if isinstance(other, SharedContext):
        return self._data != other._data
    return True


For example, a TriggerKit user might want their create a google sheet and then refrence sheet's id in a later action. This can be done by storing the sheet id in the shared context after creating the sheet, and then retrieving it in the next action.

In [ ]:
example = SharedContext()
example['sheet_id'] = '1B34VeLhAnujcnDO1Q8mcTsgDW42Q2nZBEA8mWOLFiPE'

print(example['sheet_id'])

1B34VeLhAnujcnDO1Q8mcTsgDW42Q2nZBEA8mWOLFiPE


Maybe the user also chains in a function that looks for what emails to share the sheet with and wants to store the result in the shared context as well

In [ ]:
example.emails = ['cO8dD@example.com', 'hYh0q@example.com']

print(example.emails)

['cO8dD@example.com', 'hYh0q@example.com']


We might want a way to see what value pairs are added by actions, so let's add a way to check what values a action added during it's run. This could be a simple  dictionary that get's cleared after each action run. This will allow us to see what values were added by each action, and we can use this information to debug any issues that may arise. 

In [ ]:
#| export

@patch
def updates(self: SharedContext) -> Dict[str, Any]:
    """Returns the subset of keys written during this run."""
    return dict(self._writes)

@patch
def clear_updates(self: SharedContext):
    """Clears the updates made during this run."""
    self._writes.clear()

Now, let's see if the `updates` method works. This method could be called by the action runner to update the shared context with new information. The method should take a dictionary as an argument and update the shared context with the new values.

In [ ]:
example.updates()

{'sheet_id': '1B34VeLhAnujcnDO1Q8mcTsgDW42Q2nZBEA8mWOLFiPE',
 'emails': ['cO8dD@example.com', 'hYh0q@example.com']}

That looks good! What about the clear method? This method should be called by the action runner to clear the log of what is added between runs.

In [ ]:
example.clear_updates()
print(example.updates())

{}


Perfect! Looks like things are working as expected. 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()